## numpy

In [1]:
import numpy as np

In [2]:
# N은 배치 크기이며, D_in은 입력의 차원입니다.
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.

N, D_in, H, D_out = 64, 1000, 100, 10

In [3]:
# 무작위의 입력과 출력 데이터를 생성합니다.
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

In [10]:
x.shape

(64, 1000)

In [4]:
# 무작위로 가중치를 초기화합니다.
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

In [14]:
print(
    w1.shape,
    w2.shape
)

(1000, 100) (100, 10)


In [17]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y를 계산합니다.
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)
    
    # 손실(loss)을 계산하고 출력합니다.
    loss = np.square(y_pred - y).sum()
    if t%100 == 0:
        print(t, loss)
    
    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파합니다.
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred) #벡터미분
    grad_h_relu = grad_y_pred.dot(w2.T) #벡터미분
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h) #벡터미분
    
    # 가중치를 갱신합니다.
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 1.0146681946648067e-08
100 3.453226572882835e-11
200 1.3123300121477219e-13
300 5.600151799602955e-16
400 2.653116518733606e-18


## PyTorch: Tensors

In [18]:
import torch

In [19]:
dtype = torch.float
device = torch.device('cuda:0')

In [22]:
# N은 배치 크기이며, D_in은 입력의 차원입니다.
# H는 은닉층의 차원이며, D_out은 출력 차원입니다.
N, D_in, H, D_out = 64, 1000, 100, 10

In [23]:
# 무작위의 입력과 출력 데이터를 생성합니다.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

In [24]:
#무작위로 가중치를 초기화합니다.
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

In [ ]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계: 예측값 y를 계산합니다.
    h = x.mm(w1)